# Create a Wide-Area Ethernet (Layer 2) Network: Automatic Configuration

This notebook shows how to create an isolated local Ethernet and connect compute nodes to it and use FABlib's automatic configuration functionality.

## Import the FABlib Library


In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

## Create the Experiment Slice

The following creates two nodes with basic NICs connected to an isolated WAN Ethernet.  

Two nodes are created and one NIC component is added to each node.  This example uses components of model `NIC_Basic` which are SR-IOV Virtual Function on a 100 Gpbs Mellanox ConnectX-6 PCI device. The VF is accessed by the node via PCI passthrough. Other NIC models are listed below. When using dedicated PCI devices the whole physical device is allocated to one node and the device is accessed by the node using PCI passthrough. Calling the `get_interfaces()` method on a component will return a list of interfaces. Many dedicated NIC components may have more than one port.  Either port can be connected to the network.

Automatic configuration requires specify a subnet for the network and setting the interface's mode to `auto` using the `iface1.set_mode('auto')` function before submitting the request. With automatic configuration, FABlib will allocate an IP from the network's subnet and configure the device during the post boot configuration stage.  Optionally, you can add routes to the node before submitting the request.


NIC component models options:
- NIC_Basic: 100 Gbps Mellanox ConnectX-6 SR-IOV VF (1 Port)
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports) 
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports) 

In [ ]:
slice_name = 'Project'
[site1,site2]  = ['TACC','DALL'] #fablib.get_random_sites(count=2)
print(f"Sites: {site1}, {site2}")

shared_packages = 'iperf3'
node1_packages = 'httpd wget sysstat'
client_packages = 'dnf-plugins-core epel-release python3 git'
clients_per_site = {
    site2: 5
}
client_prefix = 'client'

node1_name = 'router'
router_nic = 'NIC_ConnectX_5'
node2_name = 'client1'
network_name='edge_net'

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)
# Must use Rocky Linux 8 (default) for L2 network to be initalized properly
image = None #'default_ubuntu_20'
# Network
net1 = slice.add_l2network(name=network_name, subnet=IPv4Network("192.168.1.0/24"))

# router
router = slice.add_node(name=node1_name, site=site1,cores=16,ram=32,disk=20,image=image)
iface1 = router.add_component(model=router_nic, name='nic1').get_interfaces()[0]
iface1.set_mode('auto')
net1.add_interface(iface1)
# router.add_component(model="GPU_A30", name="video_encoder")

for site, clients in clients_per_site.items():
    for i in range(1, clients + 1):
        node2 = slice.add_node(name=client_prefix + str(i), site=site, cores=2,ram=4,disk=10,image=image)
        iface2 = node2.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
        iface2.set_mode('auto')
        net1.add_interface(iface2)
# client1
# node2 = slice.add_node(name=node2_name, site=site2, cores=2,ram=4,disk=10,image=image)
# iface2 = node2.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
# iface2.set_mode('auto')
# net1.add_interface(iface2)

#Submit Slice Request
slice.submit()
slice.wait_ssh(progress=True)

In [ ]:
for node in slice.get_nodes():
    if node.get_name() != node1_name:
        node.execute_thread("sudo dnf update -y; sudo dnf config-manager --add-repo=https://negativo17.org/repos/epel-multimedia.repo; sudo dnf config-manager --set-enabled powertools; sudo dnf install %s %s -y; sudo dnf install ffmpeg -y; git clone https://github.com/teaching-on-testbeds/AStream" % (shared_packages, client_packages))

print("Setting up router...")
# Start web server on router
router = slice.get_node(name=node1_name)
router.execute("sudo dnf update -y; sudo dnf install %s %s -y" % (shared_packages, node1_packages))
router.execute("wget https://nyu.box.com/shared/static/d6btpwf5lqmkqh53b52ynhmfthh2qtby.tgz -O media.tgz && sudo tar -v -xzf media.tgz -C /var/www/html/ && sudo systemctl enable --now httpd")



## Run the Experiment

With automatic configuration the slice is ready for experimentation after it becomes active.  Note that automatic configuration works well when saving slices to a file and reinstantiating the slice.  Configuration tasks can be stored in the saved slice, reducing the complexity of notebooks and other runtime steps.

We will find the ping round trip time for this pair of sites.  Your experiment should be more interesting!


In [ ]:
slice = fablib.get_slice(slice_name)

node1 = slice.get_node(name=node1_name)        
node2 = slice.get_node(name=node2_name)           

node2_addr = node2.get_interface(network_name=network_name).get_ip_addr()

stdout, stderr = node1.execute(f'ping -c 30 {node2_addr}')

In [ ]:
router = slice.get_node(node1_name)
routerIP = router.get_interface(network_name=network_name).get_ip_addr()
print("hello %s " % routerIP)

Change networkType to either "plain", "dpdk", or "rdma" in next cell before testing a new network type

In [ ]:
import time
router = slice.get_node(node1_name)
routerIP = router.get_interface(network_name=network_name).get_ip_addr()
networkType = "dpdk"
runs = [1,2,5]

remoteOutDir = "/home/rocky/iperf"
router.execute("mkdir -p " + remoteOutDir)
for clientsToTest in runs:
    print("Testing %d clients on %s network" % (clientsToTest, networkType))
    logFile = remoteOutDir + '/%s.iperf.%d.log' % (networkType, clientsToTest)
    router.execute('rm %s' % (logFile))
    for i in range(1, clientsToTest + 1):
        router.execute('timeout 35 iperf3 -s -p %d --logfile %s --json 2>/dev/null >/dev/null &' % (5000 + i,logFile))

    print("starting clients")

    for i in range(1, clientsToTest + 1):
        client = slice.get_node(client_prefix + str(i))
        client.execute('iperf3 -c %s -p %d --time 30 2>/dev/null >/dev/null &' % (routerIP, 5000 + i))
        print('connected to client%d' % i)
        
    remoteSarFile = "%s/%s.iperf.sar.%d.txt" % (remoteOutDir, networkType, clientsToTest)
    router.execute('sar -u 1 30 -n DEV -wr > %s 2>/dev/null &' % (remoteSarFile))
    time.sleep(35)
    router.execute('tail %s -n 15' % remoteSarFile)

In [ ]:
import os
iperfLogDir = "/home/fabric/work/project/iperf"
os.system("mkdir -p %s" % iperfLogDir)
for clientsToTest in runs:
    logfile = "%s.iperf.%d.log" % (networkType, clientsToTest)
    iperfLogPath =iperfLogDir + "/" + logfile
    router.download_file(local_file_path=iperfLogPath, remote_file_path=remoteOutDir + "/" + logfile)

In [ ]:
import json
import re
import collections.abc

for clientsToTest in runs:
    iperfLogPath = iperfLogDir + "/%s.iperf.%d.log" % (networkType, clientsToTest)
    print("Reading %s" % iperfLogPath)
    with open(iperfLogPath) as f:
        obj = json.loads("[" + f.read().replace('}\n{', '},{') + "]")
    obj = obj if isinstance(obj, collections.abc.Sequence) else [obj]

    length = 0
    totalThroughput = 0
    for clientStream in obj:
        if not ("sum_received" in clientStream["end"]):
            continue
        throughput = (clientStream["end"]["sum_received"]["bits_per_second"] / 1000000)
        totalThroughput = totalThroughput + throughput
        length += 1
        print(str(throughput) + " Mb/sec")

    avgThroughput = totalThroughput / length

    print("%d Clients Avg throughput: %s Mb/sec, Total: %s" % (clientsToTest, avgThroughput, totalThroughput))
    

In [ ]:
import os
import time
router = slice.get_node(node1_name)
routerIP = router.get_interface(network_name=network_name).get_ip_addr()
testDuration = 60
# netowrkType = "plain" | "dpdk"
# networkType = "plain"
for clientsToTest in runs:
    outDir = "/home/rocky/runs/%s/%dclient" % (networkType, clientsToTest)

    for i in range(1, clientsToTest + 1):
            client = slice.get_node(client_prefix + str(i))
            client.execute("rm -r %s" % outDir)
            client.execute("mkdir -p %s; cd %s; timeout %d python3 ~/AStream/dist/client/dash_client.py -m http://%s/media/BigBuckBunny/4sec/BigBuckBunny_4s.mpd -p 'basic' 2>/dev/null >/dev/null &" % (outDir, outDir, testDuration, routerIP))
            print("Started video streaming on client%d" % i)
    print("Waiting %d seconds..." % testDuration)
    time.sleep(testDuration)

    localVideoDir = "/home/fabric/work/project/videoStreaming/%s/%dclient" % (networkType, clientsToTest)

    def getLocalVideoCSVPath(clientIndex):
        return "%s/client%d.astream.csv" % (localVideoDir, clientIndex)

    os.system("mkdir -p " + localVideoDir)
    for i in range(1, clientsToTest + 1):
            client = slice.get_node(client_prefix + str(i))
            filepath = client.execute('find %s -type f -name "*.csv"' % outDir)[0].strip()
            client.download_file(local_file_path=getLocalVideoCSVPath(i), remote_file_path=filepath)
    print("Downloaded csv files to %s" % localVideoDir) 

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

c = {'INITIAL_BUFFERING': 'violet', 'PLAY': 'lightcyan', 'BUFFERING': 'lightpink'}

clientColors = {
    1: 'k',
    2: 'g',
    3: 'b',
    4: 'm',
    5: 'r',
}

for clientsToTest in runs:
    totalAvg = 0
    totalVariance = 0
    
    localVideoDir = "/home/fabric/work/project/videoStreaming/%s/%dclient" % (networkType, clientsToTest)

    for i in range(1, clientsToTest + 1):
        local = getLocalVideoCSVPath(i)
        print(local)
        dash = pd.read_csv(local)

        dash = dash.loc[dash.CurrentPlaybackState.isin(c.keys() )]
        states = pd.DataFrame({'startState': dash.CurrentPlaybackState[0:-2].values, 'startTime': dash.EpochTime[0:-2].values,
                                'endState':  dash.CurrentPlaybackState[1:-1].values, 'endTime':   dash.EpochTime[1:-1].values})


        for index, s in states.iterrows():
          plt.axvspan(s['startTime'], s['endTime'],  color=c[s['startState']], alpha=1)

        streaming = dash[dash.Action!="Writing"]
        bitrate = streaming.Bitrate

        avg = bitrate.mean() / 1_000_000
        variance = bitrate.var() / 1_000_000
        print("Bitrate average: %s Mbps" % (avg))
        print("Bitrate variance: %s" % (variance))
        totalAvg = totalAvg + avg
        totalVariance = totalVariance + variance

        plt.plot(streaming.EpochTime, bitrate, clientColors[i] + 'x:', label= "client" + str(i))
    plt.legend()
    plt.title("Plain Network Video rate (avg %.5f Mbps, var %.5f)" % (totalAvg, totalVariance / clientsToTest));
    # plt.title("Plain Network Video rate (avg %.5f Mbps, var %.5f)" % (avg, variance));

    plt.ylabel("Bitrate (bits/sec)")
    plt.xlabel("Time (s)");

        # print("ploted client" + str(i)) 

    plt.savefig(localVideoDir + "/bitrate.png")
    plt.show()

In [ ]:
# Install Mellanox OFED driver
router.upload_file(local_file_path="/home/fabric/work/project/MLNX_OFED_LINUX-5.8-4.1.5.0-rhel8.9-x86_64.tgz", remote_file_path="/home/rocky/MLNX_OFED_LINUX-5.8-4.1.5.0-rhel8.9-x86_64.tgz")

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
# slice.delete()